In [1]:
import networkx as nx
from prettytable import PrettyTable
import statistics

In [2]:
"""
def initial_graph_analysis(location):
    graph=nx.read_edgelist(location)
    
    print("La cantidad de nodos (grado) del grafo introducido es", nx.number_of_nodes(graph),".")
    print("La cantidad de ramas (tamaño) del grafo introducido es", nx.number_of_edges(graph),".")
    
    if nx.is_directed(graph):
        print("Se trata de un grafo dirigido.")
    else:
        print("Se trata de un grafo no dirigido.")
        
    if nx.is_connected(graph):
        print("Se trata de un grafo conexo.")
    else:
        print("Se trata de un grafo no conexo.")
    
    print("La densidad del grafo es de",nx.density(graph),".")
"""

'\ndef initial_graph_analysis(location):\n    graph=nx.read_edgelist(location)\n    \n    print("La cantidad de nodos (grado) del grafo introducido es", nx.number_of_nodes(graph),".")\n    print("La cantidad de ramas (tamaño) del grafo introducido es", nx.number_of_edges(graph),".")\n    \n    if nx.is_directed(graph):\n        print("Se trata de un grafo dirigido.")\n    else:\n        print("Se trata de un grafo no dirigido.")\n        \n    if nx.is_connected(graph):\n        print("Se trata de un grafo conexo.")\n    else:\n        print("Se trata de un grafo no conexo.")\n    \n    print("La densidad del grafo es de",nx.density(graph),".")\n'

In [2]:
def generate_random_compare(reiterations, location, analysis = False):
    
    # Se carga el grafo con el que comparar y se miden los parámetros de interés
    
    graph=nx.read_edgelist(location)
    
    grado=graph.number_of_nodes()
    tamaño=graph.number_of_edges()
    
    if analysis == True:
    
        if nx.is_directed(graph):
            directed = "Si"
        else:
            directed = "No"
    
        if nx.is_connected(graph):
            connected = "Si"
        else:
            connected = "No"
    
        y = PrettyTable(["Grado","Tamaño", "Dirigido","Conexo","Densidad"])
        y.add_row([grado, tamaño, directed, connected, nx.density(graph) ])
    
        print("Análisis básico del grafo:")
        print(y)
    
    if nx.number_of_selfloops(graph) > 0:
        graph.remove_edges_from(nx.selfloop_edges(graph))
    
    betweenness = sum(nx.betweenness_centrality(graph).values())/grado
    
    clustering = nx.average_clustering(graph)
    
    closeness = sum(nx.closeness_centrality(graph).values())/grado
    
    all_centrality = nx.degree_centrality(graph)
    centrality = sum(all_centrality.values())/nx.number_of_nodes(graph)
    
    all_cores = nx.core_number(graph)
    kcore = max(all_cores.values())
        
    density = nx.density(graph)
    
    random_count_betweenness = []
    random_count_closeness = []
    random_count_clustering = []
    random_count_centrality = []
    random_count_kcore = []
    random_count_density = []
    
    # Se realizan X grafos aleatorios y se añaden a las listas anteriores sus parámetros
    
    for _ in range(reiterations):
        
        random = nx.gnm_random_graph(grado,tamaño)
        
        random_count_betweenness.append(sum(nx.betweenness_centrality(random).values())/grado)

        random_count_closeness.append(sum(nx.closeness_centrality(random).values())/grado)
        
        random_count_clustering.append(nx.average_clustering(random))
        
        all_random_centrality=nx.degree_centrality(random)
        random_count_centrality.append(sum(all_random_centrality.values())/nx.number_of_edges(random))
        
        cores=nx.core_number(random)
        random_count_kcore.append(max(cores.values()))
        
        random_count_density.append(nx.density(random))
    
    # Se obtiene la cantidad relativa de veces que se ha igualado o superado cada parámetro
    
    p_value_betweenness = sum(map(lambda x : x >= betweenness ,random_count_betweenness))/len(random_count_betweenness)
    p_value_closeness = sum(map(lambda x : x >= closeness ,random_count_closeness))/len(random_count_closeness)
    p_value_clustering = sum(map(lambda x : x >= clustering,random_count_clustering))/len(random_count_clustering)
    p_value_centrality = sum(map(lambda x : x >= centrality,random_count_centrality))/len(random_count_centrality)
    p_value_kcore = sum(map(lambda x : x >= kcore,random_count_kcore))/len(random_count_kcore)
    p_value_density = sum(map(lambda x : x >= density,random_count_density))/len(random_count_density)
    
    # Se genera una tabla para facilidad de observación de resultados
    
    
    
    x = PrettyTable(["Parameter","Value in graph", "Value in random", "P-value","Random Graphs Generated"])
    x.add_row(["Betweenness",betweenness, statistics.mean(random_count_betweenness), p_value_betweenness, reiterations])
    x.add_row(["Closeness",closeness, statistics.mean(random_count_closeness),p_value_closeness, reiterations])
    x.add_row(["Clustering",clustering, statistics.mean(random_count_clustering), p_value_clustering, reiterations])
    x.add_row(["Centrality",centrality, statistics.mean(random_count_centrality), p_value_centrality, reiterations])
    x.add_row(["K-cores",kcore, statistics.mean(random_count_kcore), p_value_kcore,reiterations])
    
    print("\n","Comparación del grafo con grafos aleatorios")
    return x


In [3]:
print(generate_random_compare(10, "eco_edges.txt",True))

Análisis básico del grafo:
+-------+--------+----------+--------+-------------------+
| Grado | Tamaño | Dirigido | Conexo |      Densidad     |
+-------+--------+----------+--------+-------------------+
|   54  |  353   |    No    |   Si   | 0.246680642907058 |
+-------+--------+----------+--------+-------------------+

 Comparación del grafo con grafos aleatorios
+-------------+----------------------+----------------------+---------+-------------------------+
|  Parameter  |    Value in graph    |   Value in random    | P-value | Random Graphs Generated |
+-------------+----------------------+----------------------+---------+-------------------------+
| Betweenness | 0.014634736332849537 | 0.014968015911412138 |   1.0   |            10           |
|  Closeness  |  0.5733438354455607  |  0.5635747456493093  |   0.0   |            10           |
|  Clustering | 0.41278095937953413  |  0.2449680050486387  |   0.0   |            10           |
|  Centrality | 0.24458420684835783  | 0.037